In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import calendar
from scipy.stats import linregress
from pprint import pprint

# Import the API key
from api_keys import alpha_vantage_api_key, bls_api_key, fred_api_key


# https://cran.r-project.org/web/packages/eFRED/vignettes/FetchingDataFromFRED.html
# https://pypi.org/project/fredapi/
from fredapi import Fred
# Monthly Stock Profits dataFrame - 1
# Constants
stock_symbol = 'QQQ'
start_date = '2010-01-01'
end_date = '2019-12-31'
URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={alpha_vantage_api_key}&outputsize=full'


# Fetch stock data from alphavantage
response = requests.get(URL).json()

# Define an empty list to fetch the stock data
stock_data = []

for date, values in response['Time Series (Daily)'].items():
    stock_data.append({
        "Date": date,
        "Open": values['1. open'],
        "High": values['2. high'],
        "Low": values['3. low'],
        "Close": values['4. close'],
        "Volume": values['5. volume']
    })

# Conver List to dataFrame
stock_info_df = pd.DataFrame(stock_data)
stock_info_df = stock_info_df[(stock_info_df['Date'] >= start_date) & (stock_info_df['Date'] <= end_date)]

stock_info_df.reset_index()
stock_info_df['Date'] = pd.to_datetime(stock_info_df['Date'])
stock_info_df['Open'] = stock_info_df['Open'].astype(float)
stock_info_df['Close'] = stock_info_df['Close'].astype(float)

stock_info_df

In [ ]:
# Monthly Stock Profits dataFrame - 2
# Date, Month Column creation by using Date
stock_info_df['Year'] = stock_info_df['Date'].dt.year
stock_info_df['Month_Number'] = stock_info_df['Date'].dt.month

# Open price for each Year, Month & Close price for each Year, Month
monthly_data = stock_info_df.groupby(['Year', 'Month_Number'])
first_open = monthly_data['Open'].first().reset_index()
last_close = monthly_data['Close'].last().reset_index()

stock_info_df

In [ ]:
# Monthly Stock Profits dataFrame - 3 (final)

monthly_profit = last_close['Close'] - first_open['Open']

# dataFrame creation
monthly_profit_df = pd.DataFrame({
                                    'Year': first_open['Year'],
                                    'Month_Number': first_open['Month_Number'],
                                    'Monthly_Profit': monthly_profit
                                })

# Generate month numbers (1-12)
month_numbers = list(range(1, 13))

# Use list comprehension to map month numbers to month names
month_names = [calendar.month_name[month] for month in month_numbers]

# Create the DataFrame
month_df = pd.DataFrame({
                            'Month_Number': month_numbers,
                            'Month': month_names
                        })

# Merge the DataFrames on 'Month' from monthly_profit_df and 'Month Number' from month_df
monthly_profit_merge_df = pd.merge(monthly_profit_df, month_df, left_on='Month_Number', right_on='Month_Number', how='left')

monthly_profit_final_df = monthly_profit_merge_df[["Year", "Month", "Month_Number", "Monthly_Profit"]]
monthly_profit_final_df["Year"] = monthly_profit_final_df["Year"].astype('str')

monthly_profit_final_df

In [ ]:
# Initialize the FRED API key
fred = Fred(api_key=fred_api_key)
# data = fred.search('interest rate')

# Fetch data for the Federal Funds Effective Rate
series_id = 'FEDFUNDS'  # Series ID for the Federal Funds Rate
price_change_Data = fred.get_series(series_id, observation_start='2014-01-01', observation_end='2019-12-31')

# Convert the series data into a DataFrame
price_change_Data_df = pd.DataFrame(price_change_Data, columns=['Rate'])
price_change_Data_df.reset_index(inplace=True)
price_change_Data_df.columns = ['Date', 'Rate']

# Extract 'Year' and 'Month' from 'Date'
price_change_Data_df['Year'] = price_change_Data_df['Date'].dt.year
price_change_Data_df['Month'] = price_change_Data_df['Date'].dt.strftime('%B')  # Convert date to month name

# Optionally, rearrange columns if needed
price_change_Data_df = price_change_Data_df[['Year', 'Month', 'Rate']]

# Convert 'Rate' to float, handle non-numeric issues
price_change_Data_df['Rate'] = pd.to_numeric(price_change_Data_df['Rate'], errors='coerce')
price_change_Data_df["Year"] = price_change_Data_df["Year"].astype('str')


price_change_Data_df

In [ ]:
# Merge two dataFrame (Nasdaq Monthly profit + Monthly price chnage)

MonthlyClose_InterestRate_df = pd.merge(monthly_close_final_df, price_change_Data_df, on= ["Year","Month"], how="inner")

MonthlyClose_InterestRate_df

In [ ]:
# Combine 'Year' and 'Month' into a 'Date' for plotting
MonthlyClose_InterestRate_df.plot(kind='scatter',x = 'Price_change',y = 'Monthly_profit')


plt.title(f" Monthly profits vs Monthly Bitcon price change")
plt.xlabel("Monthly Price Change")
plt.ylabel("Monthly Profit")
plt.show()